In [ ]:
%matplotlib widget

import os
import numpy as np
import pandas as pd
import itertools as it
from glob import glob
import matplotlib.pyplot as plt
from scipy import interpolate
from sbpy.calib import Sun
from astropy import units as u
from astropy.io import fits

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from astropy.visualization import ImageNormalize, PercentileInterval, AsinhStretch
from astropy.visualization import simple_norm

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata, fix_lamps_metadata, get_masters_metadata
from lvmdrp.utils import metadata as md
from lvmdrp import config, log, path, __version__ as drpver

from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.core import tracemask
from lvmdrp.functions import imageMethod
from lvmdrp.functions import run_drp as drp

plt.style.use("seaborn-v0_8-talk")

In [ ]:
# _ = drp.get_master_metadata(overwrite=True)

In [ ]:
masters_flat = md.get_metadata(kind="master", tileid=1111, mjd="6014?", imagetyp="flat")
masters_flat = masters_flat.loc[masters_flat.name.str.contains("mflat")]
masters_flat.sort_values("camera", inplace=True)
masters_flat

In [ ]:
for idx, flat in masters_flat.iterrows():
    # if flat.camera != "b2": continue
    if flat.camera[0] in {"b", "r"}:
        lamp = "ldls"
    else:
        lamp = "quartz"
    mflat = path.full("lvm_master", drpver=drpver, tileid=flat.tileid, mjd=flat.mjd, camera=flat.camera, kind=f"mflat_{lamp}")
    
    # trace peak of fibers
    mtrace = path.full("lvm_master", drpver=drpver, tileid=flat.tileid, mjd=flat.mjd, camera=flat.camera, kind="mtrace")
    trace_data, trace = imageMethod.trace_peaks(in_image=mflat, out_trace=mtrace, steps=30, coadd=5, threshold=0.5, max_diff=1.5, method="gauss", median_box=10, median_cross=1, poly_disp=4, write_trace_data=True, display_plots=True)

In [ ]:
trace = fits.open(f"data_traces/traces_{camera}.fits")
trace.info()

traces = trace[0].data
mask = trace["BADPIX"].data
coeffs = trace["COEFFS"].data

x_pixels = np.arange(traces.shape[1])
y_pixels = np.arange(traces.shape[0])

ifiber = 1

plt.figure()
plt.plot(x_pixels, np.poly1d(coeffs[ifiber-1][::-1])(x_pixels))
plt.plot(x_pixels, np.poly1d(coeffs[ifiber][::-1])(x_pixels))
plt.plot(x_pixels, np.poly1d(coeffs[ifiber+1][::-1])(x_pixels))

In [ ]:
# load traces
t = tracemask.TraceMask()
t.loadFitsData(f"data_traces/traces_{camera}.fits")
# define bad fibers
bad_fibers = np.where(t._mask.sum(axis=1) == 4086)[0]

# plot some traces
x_pixels = np.arange(t._data.shape[1])
y_pixels = np.arange(t._fibers)
plt.figure()
plt.plot(x_pixels, np.polynomial.Polynomial(t._coeffs[0])(x_pixels))
plt.plot(x_pixels, np.polynomial.Polynomial(t._coeffs[1])(x_pixels))
plt.plot(x_pixels, np.polynomial.Polynomial(t._coeffs[2])(x_pixels))

# define spec fibermap
spec = fibermap[fibermap["spectrographid"] == int(camera[1])]
# get fiber positions at reference column
positions = fibermap[fibermap["spectrographid"] == int(camera[1])][f"ypix_{camera[0]}"].value

print("bad/missing fibers:")
print(spec[bad_fibers])

for column in range(t._data.shape[1]):
    
    # linearly interpolate coefficients at masked fibers
    mask = t._mask[:, column]
    for order in range(t._coeffs.shape[1]):
        t._coeffs[mask, order] = np.interp(y_pixels[mask], y_pixels[~mask], t._coeffs[~mask, order])


# evaluate trace at interpolated fibers
for ifiber in y_pixels[bad_fibers]:
    poly = np.polynomial.Polynomial(t._coeffs[ifiber, :])
    t._data[ifiber, :] = poly(x_pixels)

plt.figure()
plt.plot(x_pixels, np.polynomial.Polynomial(t._coeffs[0])(x_pixels))
plt.plot(x_pixels, np.polynomial.Polynomial(t._coeffs[1])(x_pixels))
plt.plot(x_pixels, np.polynomial.Polynomial(t._coeffs[2])(x_pixels))

In [ ]:
flat = image.loadImage(mflat)
profile = flat.getSlice(2000, axis="y")._data

pixels = np.arange(profile.size)
plt.figure(figsize=(20,5))
plt.plot(pixels, profile, "-k")
plt.plot(pixels[positions], profile[positions], "or")

In [ ]:
# from copy import deepcopy as copy

# pixels = np.arange(trace_data._data.shape[1])
# mask = (trace_data._data != 0).sum(axis=0).astype(bool)
# x_peak = copy(1 + pixels[mask])
# y_peak = copy(1 + trace_data._data[:, mask])

# x_poly = np.tile(x_peak, trace._fibers)
# y_poly = x_poly

# x_peak.shape, y_peak.shape, trace._fibers
# np.tile(x_peak, trace._fibers).shape
# y_peak.flatten().shape
# table = np.column_stack((x_poly, y_poly))

In [ ]:
# imageMethod.extract_spectra(in_image=mflat, out_rss="test_rss.fits", in_trace=mtrace, fwhm="test_fwhm.fits", parallel=0, plot_fig=False)

In [ ]:
# idx = -1
# print(masters_flat.iloc[idx].path)
# flat = image.loadImage(masters_flat.iloc[idx].path)

# yguess = flat.getSlitmap()["ypix"]

# profile = np.nan_to_num(flat._data[:, 2000], nan=0.0)
# ypix = np.arange(profile.size)

# # guess_heights = np.interp(yguess, ypix, profile)
# guess_heights = np.ones_like(yguess) * np.nanmax(profile)
# ref_profile = spectrum1d._spec_from_lines(yguess, sigma=1.2, wavelength=ypix, heights=guess_heights)

# plt.figure(figsize=(20,5))
# plt.step(ypix, ref_profile, lw=1, where="mid")
# plt.step(ypix, profile, lw=1, color="k", where="mid")
# plt.plot(yguess, np.ones_like(yguess) * np.nanmax(profile), ".", ms=5, color="red")

# plt.tight_layout()

# cc, bhat, mhat = spectrum1d._cross_match(ref_spec=ref_profile, obs_spec=profile, stretch_factors=np.linspace(0.7,1.3,5000), shift_range=[-100, 100])
# print(cc, bhat, mhat)

# plt.figure(figsize=(20,5))
# plt.step(ypix, spectrum1d._apply_shift_and_stretch(ref_profile, shift=bhat, stretch_factor=mhat), lw=0.5, color="b", where="mid")
# plt.step(ypix, profile, lw=1, color="k", where="mid")
# plt.plot(yguess*mhat + bhat, np.ones_like(yguess) * np.nanmax(profile), ".", ms=5, color="red")
# plt.tight_layout()

In [ ]:
for _, row in metadata.iterrows():
#     masters_cal = md.match_master_metadata(target_imagetyp=row.imagetyp, target_camera=row.camera)
#     bias, dark = masters_cal["bias"], masters_cal["dark"]
#     mbias_path = path.full("lvm_cal_mbias", drpver=drpver, tileid=bias.tileid, mjd=bias.mjd, camera=bias.camera)
#     mdark_path = path.full("lvm_cal_time", drpver=drpver, tileid=dark.tileid, mjd=dark.mjd, camera=dark.camera, exptime=dark.exptime, kind="mdark")
    
#     raw_path = path.full("lvm_raw", hemi="s", mjd=row.mjd, camspec=row.camera, expnum=row.expnum)
#     pre_path = path.full("lvm_anc", drpver=drpver, hemi="s", tileid=row.tileid, mjd=row.mjd, camera=row.camera, expnum=row.expnum, imagetype=row.imagetyp, kind="p")
    cal_path = path.full("lvm_master", drpver=drpver, tileid=row.tileid, mjd=row.mjd, camera=row.camera, kind="mflat")
    pks_path = path.full("lvm_cal", drpver=drpver, hemi="s", tileid=row.tileid, mjd=row.mjd, camera=row.camera, expnum=row.expnum, kind="peaks", ext="txt")
    
    # imageMethod.preproc_raw_frame(raw_path, pre_path)
    # imageMethod.detrend_frame(pre_path, cal_path)
    imageMethod.find_peaks_auto(cal_path, "peaks_test.fits", slice=2000, display_plots=True)

In [ ]:
cam = "b1"

input_fiberflat_path = f"/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/sas/sdsswork/lvm/spectro/redux/0.1.0.dev0/1111/60115/ancillary/lvm-cflat-{cam}-00000912.fits"
output_peaks_path = "peaks_test.fits"

imageMethod.find_peaks_auto(
    in_image=input_fiberflat_path, out_peaks=output_peaks_path,
    pixel_range=[10, 4000], fibers_dmin=5, threshold=0.08, slice=2000, median_box=[1, 10],
    display_plots=True)

imageMethod.trace_peaks(input_fiberflat_path, in_peaks=output_peaks_path, out_trace="trace_test.fits", poly_disp=6, threshold=20)

In [ ]:
x = fits.open("peaks_test.fits", ignore_missing_simple=True)
x[1].header

In [ ]:
_, obs_fibers, _, _ = np.genfromtxt(output_peaks_path, skip_header=1, dtype=[int, int, float, bool], unpack=True)

pixels = np.arange(4080)
obs_profile = np.zeros_like(pixels)
obs_profile[obs_fibers] = 1

plt.figure()
plt.hist(np.diff(obs_fibers), bins=50)
plt.gca().set_yscale("log")

In [ ]:
ref_sep = 5
ref_block = np.arange(0, ref_sep*36, ref_sep)
ref_block

ref_gap = 20
ref_dif = 125
nblocks = 18

ref_fibers = np.asarray([(ref_dif + i*(ref_block.max()+ref_gap)) + ref_block for i in range(nblocks)]).flatten()
ref_fibers

ref_profile = np.zeros_like(pixels)
ref_profile[ref_fibers] = 1

plt.figure(figsize=(15,5))
plt.step(pixels, ref_profile, lw=0.5, color="k", where="mid")
plt.step(pixels, obs_profile, lw=0.5, color="tab:blue", where="mid")


In [ ]:
# obs_profile = spectrum1d._apply_shift_and_stretch(ref_profile, shift=10, stretch_factor=0.9)
c, bhat, mhat = spectrum1d._cross_match(ref_profile, obs_profile, stretch_factors=np.linspace(0.8,1.2,2000), shift_range=[-100, 100], peak_num=648)
print(c, mhat, bhat)

plt.figure(figsize=(15,5))
plt.step(pixels, spectrum1d._apply_shift_and_stretch(ref_profile, shift=bhat, stretch_factor=mhat), lw=0.5, color="k", where="mid")
plt.step(pixels, obs_profile, lw=1, color="tab:red")

In [ ]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "lco_com"), ignore_cache=False).to_pandas()
frames_table = frames_table.loc[~frames_table.mjd.isin([60008,60009,60037,60038])]
# fix arc lamps metadata
fix_lamps_metadata(metadata=frames_table, lamp_names="argon neon ldls hgne xenon krypton".split())

# get master calibration metadata
masters_fiberflat = get_masters_metadata(path_pattern=out_calib_path, mjd="super", kind="mfiberflat").sort_values("camera")

In [ ]:
for _, mfiberflat in masters_fiberflat.iterrows():
    print(mfiberflat)
    
    try:
        imageMethod.findPeaksAuto_drp(
            in_image=mfiberflat.path,
            out_peaks=out_calib_path.format(mjd=mfiberflat.mjd, kind="mpeaks", camera=mfiberflat.camera, exptime=mfiberflat.exptime).replace(".fits", ".txt"),
            nfibers=638, slice=3000, threshold=0, method="hyperbolic", median_box=10, median_cross=1, display_plots=1
        )
    except ValueError:
        imageMethod.image_logger.error(f"error while detecting fibers in '{mfiberflat.path}'")
        continue

In [ ]:
# avoid edges by one or two boxes before fitting the trace
for _, mfiberflat in masters_fiberflat.iterrows():
    print(mfiberflat)
    try:
        imageMethod.tracePeaks_drp(
            in_image=mfiberflat.path,
            out_trace=out_calib_path.format(mjd=mfiberflat.mjd, kind="mtrace", camera=mfiberflat.camera, exptime=int(mfiberflat.exptime) if mfiberflat.exptime != "x" else mfiberflat.exptime),
            in_peaks=out_calib_path.format(mjd=mfiberflat.mjd, kind="mpeaks", camera=mfiberflat.camera, exptime=int(mfiberflat.exptime) if mfiberflat.exptime != "x" else mfiberflat.exptime).replace(".fits", ".txt"),
            steps=30, coadd=1, threshold=0, max_diff=2, method="hyperbolic", median_box=10, median_cross=1, plot=1,
            poly_disp=2
        )
    except np.linalg.LinAlgError:
        imageMethod.image_logger.error(f"error while tracing fiberflat '{mfiberflat.path}'")
        continue

In [ ]:
# from lvmdrp.core import tracemask
# from astropy.visualization import simple_norm

# trace_path = out_calib_path.format(mjd="super", kind="mtrace", camera="r2", exptime="x")

# trace = tracemask.TraceMask()
# trace.loadFitsData(trace_path)

# fiberflat = image.loadImage(trace_path.replace("mtrace", "mfiberflat"))

# fiberflat_data = fiberflat._data
# # fiberflat_data[fiberflat._mask] = np.nan

# norm = simple_norm(fiberflat_data, stretch="log", max_percent=95)
# fig, ax = plt.subplots(figsize=(10,10))
# ax.imshow(fiberflat_data, norm=norm, origin="lower", cmap="binary_r")

# for ifiber in range(trace._fibers):
#     fiber = trace.getSpec(ifiber)
#     ax.plot(fiber._pixels, fiber._data, color=plt.cm.rainbow(ifiber/trace._fibers), lw=1)

In [ ]:
# trace._pixels[[0,-1]]

In [ ]:
# peaks = np.loadtxt("data/lvm-super-mpeaks-r2-x.txt", skiprows=1)
# peaks

In [ ]:
# plt.figure(figsize=(20,5))
# plt.plot(peaks[:,2], np.zeros_like(peaks[:,0]), ".")

In [ ]:
# from np.polynomial import Polynomial

# # poly = Polynomial(trace._coeffs[0], domain=trace._pixels[[0-1]].tolist())

# # poly(trace._pixels), trace._data

# plt.figure(figsize=(20,5))

# icoef = 0
# mlinear = Polynomial.fit(peaks[:,2], trace._coeffs[:, icoef], deg=1)
# plt.plot(peaks[:,2], trace._coeffs[:, icoef] - peaks[:,2], ".")


In [ ]:
master_traces = get_masters_metadata(path_pattern=out_calib_path, mjd="super", kind="mtrace", exptime="x")
print(master_traces.to_string())

for _, mfiberflat in masters_fiberflat.iterrows():
    print(mfiberflat)
    master_trace = master_traces.query("camera==@mfiberflat.camera")
    try:
        mtrace = master_trace.iloc[0]
    except:
        imageMethod.image_logger.error(f"no master trace found for '{mfiberflat.path}'")
        continue
    
    imageMethod.extractSpec_drp(
        in_image=mfiberflat.path,
        out_rss=out_calib_path.format(mjd=mfiberflat.mjd, kind="xfiberflat", camera=mfiberflat.camera, exptime=int(mfiberflat.exptime) if mfiberflat.exptime != "x" else mfiberflat.exptime),
        in_trace=mtrace.path,
        method="aperture", aperture=4, plot=1, parallel="auto"
    )